In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [8]:
num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [9]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)
train_dataset = torchvision.datasets.CIFAR10(root='./data',train=True,
                                             download=True,transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data',train=False,
                                             download=True,transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,
                                           shuffle=False)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

SyntaxError: invalid syntax (3087744793.py, line 13)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

In [ ]:
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),learning_rate=learning_rate)
n_total_step = len(train_loader)
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs,labels) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1)%2000==0:
            print(f'Epoch[{epoch+1}/{num_epochs}],Step[{i+1}/{n_total_step}],
                  Loss:{loss.item():.4f}')
print('finish training')

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(output)
        n_samples += labels.size(0)
        n_correct += (predicted==labels).sum().item()
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if(label==pred):
                n_class_correct[label]+=1
            n_class_samples[label]+=1
    acc = 100.0*n_correct/n_samples
    print(f'Accuracy of {classes[i]}:{acc}%')